In [30]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import dlc_pratical_prologue as prologue

## Data set


In [18]:
N = 10
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [19]:
# Double checking
print("Training Set")
print(train_input.size())
print(train_target.size())
print(train_target.size())
print("-------------------")
print("Testing Set")
print(test_input.size())
print(test_target.size())
print(test_classes.size())

Training Set
torch.Size([10, 2, 14, 14])
torch.Size([10])
torch.Size([10])
-------------------
Testing Set
torch.Size([10, 2, 14, 14])
torch.Size([10])
torch.Size([10, 2])


## Helping functions

In [151]:
"""
Input :-

model : Pytorch NN model
input_data : Tensor of N X 2 X 14 X 14
target_data : Tensor of N X 1
batch_size : Size of the batch

-----------------------------------

Output :-

The number of samples not well classified
"""
def compute_erros(model,
                  input_data,target_data,
                  batch_size = 100):
  nb_errors = 0
  for inputs,targets in zip(input_data.split(batch_size),target_data.split(batch_size)):
    outputs = model(inputs)
    outputs = output.narrow(dim=1,start=0,length=1)
    outputs = torch.ge(output,0.5).float()
    for k in range(len(targets)):
            if outputs[k] != targets[k]:
                nb_errors += 1
    return nb_errors

"""
Input :-

model : Pytorch NN model
train_input : Tensor of N x 2 x 14 x 14
train_target : Tensor of N x 1
train_classes : Tensor of N x 2

test_input : Tensor of N x 2 x 14 x 14
test_target : Tensor of N x 1
test_classes : Tensor of N x 2

epochs : the number of passes of the entire training dataset
eta : learning parameter

batch_size : Size of the batch

-----------------------------------

Output :-

The number of samples not well classified

"""
def training_model(model,
                   train_input,train_target,train_classes,
                   test_input,test_target,test_classes,
                   eta,epochs = 25,
                   batch_size = 50):
  
  train_acc = []
  test_acc = []

  # define criterion and optimizer
  # need to check the other possibilities
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr = eta)

  for e in range(0,epochs):
    for inputs,target in zip(train_input.split(batch_size),test_target.split(batch_size)):

      outputs = model(inputs)
      #checking outputs shape
      loss = criterion(outputs,target)
      acc_loss = acc_loss + loss.item()

      # optimising parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step() 

    train_errors = compute_nb_errors(model, train_input, train_target)
    print(f"Epoch # {e+1} / Train errors: {train_errors:.2f}")
     


  return True
  



## Models